In [697]:
import random
from pulp import *
from collections import deque
import time
from collections import Counter
import networkx as nx
import math


# Ucitavanje grafova

In [698]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph

def loadGraph(input_file):
    global d
    G = nx.Graph()
    for j in range(0,n):
        G.add_node(j)

    f = open(input_file, "r")
    string = f.readline()
    string = f.readline()
    string = f.readline()
    for i in range(0, m):
        string = f.readline()
        string = string.split()
        j = int(string[0])-1
        k = int(string[1])-1
        G.add_edge(j, k)
    f.close()
    d = []
    j = 1
    for i in range(n):
        d.append([float("inf")]*(j))
        j+=1
    for i in range(G.number_of_nodes()):
        ds = nx.single_source_shortest_path_length(G, i)
        for key in ds:
            if i >= key:
                d[i][key] = ds[key]
            else:
                d[key][i] = ds[key]
    print("n: " + str(G.number_of_nodes()))
    print("m: " + str(G.number_of_edges()))
    print("ncc: " + str(nx.number_connected_components(G)))


In [699]:
def all_pairs_distance_matrix(graph):
    """
    Calculate all-pairs shortest distances using BFS for each node.
    :param graph: dict where keys are nodes (1 to n) and values are lists of neighbors
    :return: 2D list (matrix) where element [i][j] is the shortest distance from node i+1 to j+1
    """
    nodes = sorted(graph.keys())  # Sortiraj čvorove kako bi odgovarali indeksima matrice
    n = len(nodes)
    distance_matrix = [[math.inf] * n for _ in range(n)]  # Inicijalizuj matricu sa beskonačnostima

    for start in nodes:
        visited = {node: False for node in nodes}
        d_local = {node: math.inf for node in nodes}
        q = deque()
        d_local[start] = 0
        visited[start] = True
        q.append(start)

        while q:
            current = q.popleft()
            for neighbor in graph.get(current, []):
                if not visited[neighbor]:
                    visited[neighbor] = True
                    q.append(neighbor)
                    d_local[neighbor] = d_local[current] + 1

        # Popuni red u matrici udaljenosti za čvor `start`
        for node in nodes:
            distance_matrix[start - 1][node - 1] = d_local[node]

    return distance_matrix

def valid_burning_sequence(s, n, d):
    """
    Validate if the given burning sequence is valid for a graph of size n.
    :param s: List of burning nodes (sequence of nodes).
    :param n: Number of nodes in the graph.
    :param d: Distance matrix (2D list) where d[i][j] represents the shortest distance between node i+1 and j+1.
    :return: True if the sequence is valid, otherwise False.
    """
    counter = 0

    for i in range(n):
        b = len(s)
        for j in range(b):
            if d[i][s[j] - 1] <= b - (j + 1):  # Adjust index for 1-based node numbering
                counter += 1
                break

    return counter == n

# Generisanje nasumicnih burning sekvenci

In [700]:

# Učitavanje grafa
#filename = "grafovi/manji/karate.mtx"
#filename = "grafovi/manji/chesapeake.mtx"
#filename = "grafovi/manji/dolphins.mtx"
#filename = "grafovi/manji/rt-retweet.mtx"
#filename = "grafovi/manji/polbooks.mtx"
#filename = "grafovi/manji/adjnoun.mtx"
#filename = "grafovi/manji/ia-infect-hyper.mtx"
#filename = "grafovi/manji/C125-9.mtx"
#filename = "grafovi/manji/ia-enron-only.mtx"
#filename = "grafovi/manji/c-fat200-1.mtx"
#filename = "grafovi/manji/c-fat200-2.mtx"
#filename = "grafovi/manji/c-fat200-5.mtx"
#filename = "grafovi/manji/sphere.mtx"
#filename = "grafovi/manji/DD244.mtx"
#filename = "grafovi/manji/ca-netscience.mtx"
#filename = "grafovi/manji/infect-dublin.mtx"
#filename = "grafovi/manji/bio-diseasome.mtx"
#filename = "grafovi/manji/web-polblogs.mtx"
#filename = "grafovi/manji/DD687.mtx"
#filename = "grafovi/manji/rt-twitter-copen.mtx"
#filename = "grafovi/manji/ia-crime-moreno.mtx"
#filename = "grafovi/manji/soc-wiki-Vote.mtx"
#filename = "grafovi/manji/DD349.mtx"
#filename = "grafovi/manji/DD497.mtx"
#filename = "grafovi/manji/socfb-Reed98.mtx"
filename = "grafovi/manji/delaunay_n10.mtx"
#filename = "grafovi/manji/email-univ.mtx"
#filename = "grafovi/manji/econ-mahindas.mtx"
#filename = "grafovi/manji/ia-fb-messages.mtx"
#filename = "grafovi/manji/bio-yeast.mtx"
#filename = "grafovi/manji/tech-routers-rf.mtx"
#filename = "grafovi/manji/facebook.mtx"
#filename = "grafovi/manji/squirrel.mtx"

graph = load_graph_from_mtx(filename)




In [701]:


def bfs_shortest_paths(graph, start):
    dist = {node: float('inf') for node in graph} 
    dist[start] = 0
    queue = deque([start])
    paths = {node: [] for node in graph}
    paths[start] = [start]
    
    while queue:
        current = queue.popleft()
        
        for neighbor in graph[current]:
            if dist[neighbor] == float('inf'):
                dist[neighbor] = dist[current] + 1
                queue.append(neighbor)
            
            if dist[neighbor] == dist[current] + 1:
                paths[neighbor].append(current)
    
    return dist, paths

def calculate_neighbors(graph):
    neighbors_count = {node: len(graph[node]) for node in graph}
    return neighbors_count

def choose_initial_node(graph, neighbors_count):
    """
    Kombinuje heuristički i nasumični izbor početnog čvora.
    """
    top_k = int(len(graph)/3)
    # Sortiraj čvorove po broju suseda
    sorted_nodes = sorted(graph.keys(), key=lambda node: neighbors_count[node], reverse=True)
    
    # Uzmi top_k čvorova (ili manje ako ih nema dovoljno)
    top_candidates = sorted_nodes[:top_k]
    
    # Nasumično izaberi jednog od kandidata
    return random.choice(top_candidates)

def greedy_burning(graph):
    S = []  # sekvenca zapaljenih čvorova
    B = set()  # skup zapaljenih čvorova
    NB = set(graph.keys())  # skup nezapaljenih čvorova, inicijalno su to svi čvorovi grafa
    
    neighbors_count = calculate_neighbors(graph)

    current_node = random.choice(list(NB))
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)
    
    while len(B) < len(graph):
        # Korak 2: zapaliti sve susjede (koji nisu zapaljeni) od čvorova u skupu zapaljenih
        newlyburned = []
        for node in B:
            neighbors = graph[node]  # svi susjedi trenutnog zapaljenog čvora
            for neighbor in neighbors:
                if neighbor in NB:  # ako susjed nije zapaljen
                    newlyburned.append(neighbor)  # dodajemo ga u zapaljene
                    NB.remove(neighbor)  
        
        for new in newlyburned:
            B.add(new)  # dodajemo sve susjede u zapaljene
        
        # Korak 3: pronaći idući čvor koji će biti direktno zapaljen
        max_neighbors = -1
        best_node = None
        candidates_for_best = []

        
# Korak 3: pronaći idući čvor koji će biti direktno zapaljen
        if NB:
    # Napravimo listu verovatnoća proporcionalnu broju suseda
            weights = []
            for node in NB:
                notburned_neighbors = len([n for n in graph[node] if n not in B])
                weights.append(notburned_neighbors + 1)  # Dodajemo 1 da ne bismo imali težinu 0
    
    # Normalizacija težina
            total_weight = sum(weights)
            probabilities = [w / total_weight for w in weights]

    # Izbor sledećeg čvora na osnovu ponderisanih verovatnoća
            best_node = random.choices(list(NB), weights=probabilities, k=1)[0]

            S.append(best_node)
            B.add(best_node)
            NB.remove(best_node)
        else:
            break

            

    return S


def generate_multiple_random_burning_sequences_with_greedy(graph, num_sequences):
    """
    Generisanje više sekvenci sagorevanja sa smanjenom nasumičnošću.
    """
    sequences = []
    for _ in range(num_sequences):
        sequence = greedy_burning(graph)
        sequences.append(sequence)
    return sequences



burning_sequences = generate_multiple_random_burning_sequences_with_greedy(graph, 30)
for seq in burning_sequences:
    print(seq, "len:", len(seq))


[360, 817, 489, 992, 743, 978, 593, 958, 733, 729, 711] len: 11
[544, 416, 613, 147, 205, 42, 1001, 979, 713, 327, 231, 937, 927] len: 13
[800, 1020, 477, 583, 125, 707, 942, 375, 411, 434, 343] len: 11
[303, 789, 116, 894, 958, 96, 690, 439, 91, 735, 917] len: 11
[384, 951, 481, 411, 962, 468, 261, 770, 854, 216, 204, 714, 711] len: 13
[1016, 899, 548, 914, 326, 411, 104, 289, 457, 512, 181, 737] len: 12
[457, 1017, 58, 504, 713, 564, 627, 13, 87, 811, 601] len: 11
[699, 228, 272, 161, 650, 10, 470, 776, 537, 1010, 849, 791] len: 12
[400, 486, 948, 138, 808, 880, 423, 782, 184, 56, 61, 1006] len: 12
[71, 961, 77, 724, 261, 548, 164, 902, 476, 500, 923, 815] len: 12
[850, 590, 410, 684, 947, 348, 236, 878, 223, 523, 499] len: 11
[664, 79, 505, 673, 851, 187, 396, 197, 34, 208, 790] len: 11
[22, 407, 727, 516, 457, 64, 764, 159, 837, 658, 810, 43] len: 12
[650, 974, 636, 798, 566, 699, 30, 703, 189, 430, 498, 461] len: 12
[801, 1001, 240, 613, 39, 486, 225, 709, 105, 407, 283, 421, 419]

In [702]:
def transpose_list_of_lists(input_list):
    """
    Transponuje listu listi tako da prvi elementi svih podlisti budu u prvoj listi, drugi u drugoj, itd.
    """
    transposed = []
    for item in zip(*input_list):
        unique_item_list = list(set(item))
        transposed.append(unique_item_list)
    return transposed

burning_sequences_t = transpose_list_of_lists(burning_sequences)
print(burning_sequences_t)


[[384, 650, 11, 526, 400, 661, 22, 664, 927, 544, 800, 801, 303, 304, 947, 699, 197, 71, 457, 717, 209, 850, 248, 359, 360, 232, 238, 1013, 1016, 762], [899, 404, 789, 407, 279, 285, 416, 817, 951, 60, 574, 961, 449, 964, 582, 73, 590, 79, 974, 337, 607, 228, 486, 872, 1001, 745, 235, 749, 1017, 1020], [906, 272, 784, 410, 28, 416, 548, 425, 556, 948, 310, 58, 571, 831, 77, 339, 723, 727, 477, 989, 351, 481, 613, 489, 240, 369, 116, 372, 505, 636], [896, 516, 138, 908, 654, 914, 147, 149, 666, 411, 798, 543, 161, 673, 684, 945, 583, 339, 724, 980, 726, 474, 607, 992, 868, 613, 999, 504, 1018, 894], [261, 650, 660, 148, 550, 39, 808, 297, 433, 947, 566, 310, 445, 958, 317, 962, 326, 711, 713, 457, 205, 851, 96, 994, 230, 743, 884, 892, 125, 127], [10, 12, 18, 411, 923, 548, 420, 42, 564, 693, 951, 440, 187, 699, 317, 64, 707, 459, 77, 463, 978, 468, 348, 96, 226, 485, 486, 880, 761, 126], [261, 901, 391, 396, 524, 662, 407, 542, 30, 164, 423, 942, 690, 435, 180, 566, 952, 829, 197, 70, 

# PuLP

In [ ]:
    
def ILP(L,k):
    global d  
    
    #model
    model = LpProblem("ILP_COV", LpMinimize)
    
    # Definišemo varijable
  

# Definišemo b[j] varijable
    b = [LpVariable("b_{}".format(j+1), cat='Binary') for j in range(n)]
    x = [[LpVariable("x_{}_{}".format(r+1, i+1), cat='Binary') for i in range(n+1)] for r in range(k)]
    
    s1= 0
    for i in range(k):
        for j in range(n+1):
            s1 += x[i][j]
    model+=s1
        
    
    #domensko ogranicenje
    for i in range(int(k/2)):
        model += lpSum(x[i][j] for j in burning_sequences_t[i]) == 1

    
    #ogranicenja
    #(13)
    for i in range(k):
        suma=0
        for j in range(n):
            suma+=x[i][j]
        model +=(suma<=1)
        
    #(14)
    for j in range(n):
        suma = 0
        for i in range(k):
            for kk in range(n):
                if kk >= j:
                    if d[kk][j] <= i:
                        suma += x[i][kk]
                else:
                    if d[j][kk] <= i:
                        suma += x[i][kk]
        model +=(b[j]<=suma)
        
    #(12)
    for i in range(k):
        sum2 = 0
        if i == 0:
            sum1 = 1
        else:
            sum1 = 0
        for j in range(n):
            sum2 += x[i][j]
            if i >= 1:
                sum1 += x[i-1][j]
        model +=(sum2<=sum1)

    
    #(15)
    model +=lpSum(b[j] for j in range(n)) == n 
    
    
    # rjesavanje
    model.solve(PULP_CBC_CMD(maxSeconds=600, msg=True, fracGap=0))
    '''for i in range(k):
        for j in range(n+1):
            if x[i][j].value() == 1:
                print(f"x[{i}, {j}] = 1")
            else:
                print(f"x[{i}, {j}] = 0")'''


    if LpStatus[model.status] == 'Optimal':
        print(f"Optimal Solution Found: {model.objective.value()}")
        for i in range(k):
            for j in range(n):
                if x[i][j].value() == 1:
                    print(f"x[{i}, {j}] = 1")
    else:
        print("No optimal solution found")
    s = []
    
    x_out = [[0]*n for i in range(k)]
    for r in range(k):
        i = 0
        for e in x_out[r]:
            if e == 1:
                s.append(i)
                break
            i += 1
 
    s.reverse()
   

    return s
        
def main(ni, mi, input_file, Li, Ui):
    n = ni
    m = mi
    U = Ui
    L = Li
    loadGraph(input_file)
    s = ILP(L,U)

if __name__ == "__main__":
    bs = []
    n = 0
    m = 0
    d = {}
    bs_size = float("inf")
    folder_dataset = 'grafovi/manji/'
    dataset = [
        #['karate.mtx',34,78,2,3] #['dolphins.mtx',62,159,2,4],
        #['chesapeake.mtx',39,170,1,3], # instance, n, m, l, h
        #['dolphins.mtx',62,159,2,4],
        #['rt-retweet.mtx',96,117,2,5],
        #['polbooks.mtx',105,441,2,4],
        #['adjnoun.mtx',112,425,2,4],
        #['ia-infect-hyper.mtx',113,2196,1,3],
        #['C125-9.mtx',125,6963,1,3],
        #['ia-enron-only.mtx',143,623,2,4],
        #['c-fat200-1.mtx',200,1534,3,7],
        #['c-fat200-2.mtx',200,3235,2,5],
        #['c-fat200-5.mtx',200,8473,1,3],
        #['sphere.mtx',258,1026,3,7],
        #['DD244.mtx',291,822,4,7],
        #['ca-netscience.mtx',379,914,3,6],
        #['infect-dublin.mtx',410,2765,2,5],
        #['bio-diseasome.mtx',516,1188,5,7],
        #['web-polblogs.mtx',643,2280,3,5],
        #['DD687.mtx',725,2600,4,8],
        #['rt-twitter-copen.mtx',761,1029,3,7],
        #['ia-crime-moreno.mtx',829,1475,3,7],
        #['soc-wiki-Vote.mtx',889,2914,3,6],
        #['DD349.mtx',897,2087,6,12],
        #['DD497.mtx',903,2453,6,11],
        #['socfb-Reed98.mtx',962,18812,2,4],
        ['delaunay_n10.mtx',1024,3056,4,9],
        #['email-univ.mtx',1133,5451,2,5],
        #['econ-mahindas.mtx',1258,7513,2,5],
        #['ia-fb-messages.mtx',1266,6451,2,5],
        #['bio-yeast.mtx',1458,1948,4,9],
        #['tech-routers-rf.mtx',2113,6632,3,6],
        #['facebook.mtx',4039,88234,2,4],
        #['squirrel.mtx',5201,198493,2,6],
       
        ]
    for i in range(len(dataset)):
        print("________________________________________________")
        instance = dataset[i][0]
        print("instance: " + instance)
        n = dataset[i][1]
        m = dataset[i][2]
        L = dataset[i][3]
        U = dataset[i][4]
        startTime = time.time()
        main(n, m, folder_dataset + dataset[i][0], L, U)
        print("--- %s sekundi ---" % (time.time() - startTime))
    
    

________________________________________________
instance: delaunay_n10.mtx
n: 1024
m: 3056
ncc: 1
